In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
from functions import read_data

In [3]:
from config import list_cols_with_na

In [4]:
df_train, df_test = read_data()